In [ ]:
import serial
import pandas as pd


# Lista para almacenar los últimos 10 criterios
criterios_previos = []
datos_almacenados = []
n = 10
header = True  # Controla si se escribe el encabezado

if len(datos_almacenados) == 0:
    archivo = input("Ingrese el nombre del archivo .csv para guardar sus datos")

prueba = input("Ingrese el tipo de prueba que desea realizar: (1) Excentricidad, (2): Repetibilidad, (3): Error de Indicación, (4): Otro:")

prueba_esc = ''

if prueba == '1':
    prueba_esc = 'Excentricidad'
elif prueba == '2':
    prueba_esc = 'Repetibilidad'
elif prueba == '3':
    prueba_esc = 'Error_Indicación'
else: 
    prueba_esc = 'Otro'

archivo = archivo + '_' + prueba_esc + '.csv'

# Configurar el puerto serial
ser = serial.Serial(
    port='COM3',
    baudrate=9600,
    timeout=1 
)

# Dato formato al valor obtenido
def formato(datos):
    parts = datos.split()

    if len(parts) == 1:
        return 0, 'X'

    # Extraer el valor numérico, el último dígito y el criterio
    value_part = parts[0].split('/')[0]     # Parte principal del valor
    last_digit = parts[0].split('/')[1]      # Último dígito
    criteria = parts[2]                      # S o U

    if 'g' in value_part:
        formatted_value = 0

    else:
        # Convertir y combinar el valor en el formato deseado
        formatted_value = f"{float(value_part):.3f}{last_digit}"  # Forma ###.####



    return float(formatted_value), criteria

while True:
    if ser.in_waiting > 0:  # Si hay datos disponibles en el puerto

        data = ser.readline().decode('ascii', errors='ignore').strip()  # Leer la línea de datos
        val = formato(data)
        
        # Agregar el criterio actual a la lista
        criterios_previos.append(val[1])

        # Mantener solo los últimos n elementos
        if len(criterios_previos) > n:
            criterios_previos.pop(0)

        # Verificar si todos los últimos n criterios son 'S'
        if len(criterios_previos) == n and all(c == 'S' for c in criterios_previos) and val[0] > 0.001:

            ser.close() # Cerrar puerto Serial

            # Solicitar confirmación del usuario para guardar el dato
            if val[1] == 'S':
                criterio = 'Estable'
            respuesta = input(f"Dato recibido: {val[0]} g, criterio: {criterio}. ¿Deseas guarndarlo? (s/n) (Q: Salir): ")

            if respuesta.lower() == 's':
                # Guardar el valor si el usuario confirma
                datos_almacenados.append(val[0])
                print(f"Dato guardado: {val[0]} g")

                # Crear un DataFrame con el dato nuevo
                nuevo_dato = pd.DataFrame([val[0]], columns=[f'Dato (g) | {prueba_esc}'])
                
                # Guardar el nuevo dato en el archivo CSV
                nuevo_dato.to_csv(archivo, mode='a', index=False, header=header)
                header = False  # Después del primer dato, evitar escribir el encabezado de nuevo
            
            elif respuesta.lower() == 'q':
                break

            else:
                print('Dato Descartado')
            
            # Limpiar la lista de criterios para reiniciar el proceso
            criterios_previos.clear()

            # Reabrir el puerto serial

            ser.open()


Dato guardado: 100.0036 g
Dato guardado: 100.0035 g
Dato guardado: 100.0035 g
Dato guardado: 100.0036 g


In [3]:
ser.close()